# Importing Libraries 

In [1]:
import pandas as pd
import numpy as np
import sklearn

# Visualization Libraies
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix

# to split the dataset into test and train 
from sklearn.model_selection import train_test_split

# For Custom Transformer
from sklearn.base import BaseEstimator, TransformerMixin

# Transformers for the Pipeline
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
import category_encoders
from category_encoders import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Neural Networks
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

# Loading in the Dataset 
source: https://www.kaggle.com/kemical/kickstarter-projects/notebooks?sortBy=dateRun&group=upvoted&pageSize=20&datasetId=4104

In [2]:
# To view all the columns 
pd.set_option('display.max_columns', None)

df = pd.read_csv('kickstarter.csv', 
                parse_dates=['deadline', 'launched'])
df.head(5)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


Ideas:
- delete ID
- feature engineering (deadline and launched)
- Consider top five countries
- Only consider failed or successful and make it a binary classifictaion
- Will keep all the 15 main categories
- For category we could pass it to the ordinal encoding 

# EDA

In [3]:
total_nan = df.isna().sum().sort_values(ascending=False)
percentage_nan = (total_nan / df.shape[0]) * 100
tabel = pd.concat([total_nan, percentage_nan], axis=1, keys=['Total NAN', 'Percentage of NAN'])
tabel

,Total NAN,Percentage of NAN
usd pledged,3797,1.002744
name,4,0.001056
usd_goal_real,0,0.000000
usd_pledged_real,0,0.000000
country,0,0.000000
backers,0,0.000000
state,0,0.000000
pledged,0,0.000000
launched,0,0.000000
goal,0,0.000000


In [4]:
df.state.value_counts()

failed        197719
successful    133956
canceled       38779
undefined       3562
live            2799
suspended       1846
Name: state, dtype: int64

In [5]:
# filtering out the dataset for binary target variable - failed / successful
df = df.loc[(df['state'] == 'failed') | (df['state'] == 'successful')]
df.state.value_counts()

failed        197719
successful    133956
Name: state, dtype: int64

In [6]:
# Imbalance classification but not worrisome
successful = 131490/(192871+131490)
successful

0.4053816580908309

In [7]:
# changing the target variable to 0 and 1 
df['state'] = df['state'].map({'failed':0, 'successful':1})
df['state'].value_counts()

0    197719
1    133956
Name: state, dtype: int64

# Checking for leakage

In [8]:
df.loc[df['usd_pledged_real'] < df['usd_goal_real']]['state'].value_counts()

0    197713
1         5
Name: state, dtype: int64

In [9]:
# checking for backers
df.loc[df['backers'] > 70]['state'].value_counts()

1    67142
0     9151
Name: state, dtype: int64

In [10]:
df.loc[df['usd_pledged_real'] > 25000]['state'].value_counts()

1    19463
0     1374
Name: state, dtype: int64

In [11]:
df.loc[df['usd_pledged_real'] > 25000].shape

(20837, 15)

There is clearly a data leakage here. 

## Deleting verticals that we don't need 
- Dropping USD pledged and pledged as usd_pledged_real has the same information
- Dropping country as currency and country are highly correlated
- Dropping goal as we have goal converted in USD as a vertical
- Dropping backers and 

In [12]:
df = df.drop(['usd pledged', 'pledged', 'country', 'goal', 'ID', 'usd_pledged_real', 'backers'], axis=1) 
df.shape

(331675, 8)

In [13]:
# dropping 4 nan values in name
df.dropna(axis=0, inplace=True)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 331672 entries, 0 to 378660
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   name           331672 non-null  object        
 1   category       331672 non-null  object        
 2   main_category  331672 non-null  object        
 3   currency       331672 non-null  object        
 4   deadline       331672 non-null  datetime64[ns]
 5   launched       331672 non-null  datetime64[ns]
 6   state          331672 non-null  int64         
 7   usd_goal_real  331672 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int64(1), object(4)
memory usage: 22.8+ MB


## Checking high cardinality 

In [15]:
cols = ['category', 'main_category', 'currency']

for col in cols:
    print(f"{col} has {df[col].nunique()} unique values")

category has 159 unique values
main_category has 15 unique values
currency has 14 unique values


In [16]:
# Either we can delete category vertical or ordinal encode it!
df.category.value_counts()

Product Design     18680
Documentary        14523
Music              12633
Tabletop Games     11744
Shorts             11394
                   ...  
Residencies           68
Letterpress           46
Chiptune              33
Literary Spaces       19
Taxidermy             10
Name: category, Length: 159, dtype: int64

In [17]:
# Lets consider top 5 currencies and delete the rest 
df.currency.value_counts()

USD    261509
GBP     29475
EUR     14378
CAD     12375
AUD      6621
SEK      1510
MXN      1411
NZD      1274
DKK       929
CHF       652
NOK       584
HKD       477
SGD       454
JPY        23
Name: currency, dtype: int64

In [18]:
# Filtering out top 5 currencies 
df = df.loc[(df['currency'] == 'USD') | (df['currency'] == 'GBP') | (df['currency'] == 'EUR') \
      | (df['currency'] == 'CAD') | (df['currency'] == 'AUD')]

In [19]:
# changing the currency names into countries so it will be easier to ask a user for input
df['currency'] = df['currency'].map({
                'USD':'USA',
                'GBP':'UK',
                'EUR':'Europe',
                'CAD':'Canada',
                'AUD': 'Australia'})
df.currency.value_counts()

USA          261509
UK            29475
Europe        14378
Canada        12375
Australia      6621
Name: currency, dtype: int64

In [20]:
# reset index as we deleted a few rows when we filtered
df.reset_index(drop=True, inplace=True)

# Feature Engineering 
- calculating the length of the campaign from deadline and launched 
- calculating the total number of words in name 

In [21]:
# making a new column length_days
df['length_days'] = (df['deadline'] - df['launched']).dt.days + 1

# deleting deadline and launched cols
df = df.drop(['deadline', 'launched'], axis=1)

In [22]:
# Total number of words in each row in name
df['name'] = df['name'].str.split().str.len()

# Baseline Model 

In [23]:
# we need our model to beat this score
baseline = 1 - successful
baseline

0.5946183419091691

# Split Dataset

In [24]:
# Feature Matrix and Target Variable
X = df.drop('state', axis=1)
y = df['state']

In [25]:
# Splitting the dataset into train and test dataset 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=105)

In [26]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(259486, 6)
(64872, 6)
(259486,)
(64872,)


In [27]:
X_train.head()

,name,category,main_category,currency,usd_goal_real,length_days
87131,3,Technology,Technology,USA,8000.00,20
181947,3,Documentary,Film & Video,Canada,47824.01,15
86969,11,Art,Art,USA,25000.00,31
149100,3,Nonfiction,Publishing,Australia,2816.37,35
114517,1,Documentary,Film & Video,UK,32037.42,41


# Pipeline 

In [28]:
# looking at numerical attributes for simple imputer with median 
num_attribs = X_train.select_dtypes(exclude='object')
num_attribs.columns

Index(['name', 'usd_goal_real', 'length_days'], dtype='object')

In [29]:
# looking at categorical attributes for simple imputer with 'most_frequent'
cat_attribs = X_train.select_dtypes(include='object')
cat_attribs.columns

Index(['category', 'main_category', 'currency'], dtype='object')

In [30]:
# making seperate list of cols for ohe and ordinal encoding
cat_attribs_ohe = X_train[['main_category', 'currency']]
cat_attribs_ohe.columns

Index(['main_category', 'currency'], dtype='object')

In [31]:
cat_attribs_ord = X_train[['category']]
cat_attribs_ord.columns

Index(['category'], dtype='object')

In [32]:
"""Building a custom Transformer that will give the output in a dataframe after 
applying the simple imputer so we could pass it to the categorical_encoders 
which does not accept np.array""" 

class ImputerDF(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.imputer = SimpleImputer(strategy='most_frequent')
        self.cols = []
        
    def fit(self, X, y=None):
        self.imputer.fit(X)
        self.cols = list(X.columns)
        return self
    
    def transform(self, X):
        X_t = self.imputer.transform(X)
        return pd.DataFrame(X_t, columns=self.cols)

In [33]:
# Using median as the strategy for Simple Imputer to predict NaN values
num_pipeline = make_pipeline(
                            SimpleImputer(strategy='median'),
                            StandardScaler()
)
""" Adding the Custom Transformer to impute using 'most_frequent' strategy
 and giving out an output as a dataframe instead of an array """
cat_pipeline = make_pipeline(
                            ImputerDF(),
                            OrdinalEncoder(cols = cat_attribs_ord),
                            OneHotEncoder(cols = cat_attribs_ohe)
)

In [34]:
# generating a list of categorical and numerical columns to pass it in the column transformer
cat_attributes = list(cat_attribs)
num_attributes = list(num_attribs)

# putting two pipelines together using ColumnTransformer
final_pipeline = ColumnTransformer([
                            ('num_pipeline', num_pipeline, num_attributes),
                            ('cat_pipeline', cat_pipeline, cat_attributes)
])

In [35]:
# Fit and trasnform on X_train
X_train_transformed = final_pipeline.fit_transform(X_train)


In [36]:
print(X_train_transformed.shape)

(259486, 24)


# Model Selection

- Ran a few simple models and picked the most generalized model
- Using **Accuracy** as the evaluation metrics as the dataset is balanced.

## ANN - Artificial Neural Networks

In [38]:
X_train.columns

Index(['name', 'category', 'main_category', 'currency', 'usd_goal_real',
       'length_days'],
      dtype='object')

In [66]:
def create_model():
    # Initializing the ANN 
    model = Sequential([
        (Dense(units=64, kernel_initializer='he_normal', activation='relu', input_dim=24)),
        (Dense(units=54, kernel_initializer='he_normal', activation='relu')),
        (Dense(units=38, kernel_initializer='he_normal', activation='relu')),
        (Dense(units=28, kernel_initializer='he_normal', activation='relu')),
        (Dense(units=16, kernel_initializer='he_normal', activation='relu')),
        (Dense(units=8, kernel_initializer='he_normal', activation='relu')),
        (Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid'))
    ])
    
    model.compile(optimizer='nadam', loss='binary_crossentropy', metrics= ['accuracy'])
    
    return model

In [69]:
model = create_model()
model.fit(X_train_transformed, y_train, validation_split=.33, batch_size=80, epochs=50)

Epoch 1/50
1739/1739 [==============================] - 1s 797us/step - loss: 0.6536 - accuracy: 0.6318 - val_loss: 0.6339 - val_accuracy: 0.6413
Epoch 2/50
1739/1739 [==============================] - 1s 738us/step - loss: 0.6242 - accuracy: 0.6453 - val_loss: 0.6171 - val_accuracy: 0.6503
Epoch 3/50
1739/1739 [==============================] - 1s 729us/step - loss: 0.6200 - accuracy: 0.6486 - val_loss: 0.6150 - val_accuracy: 0.6509
Epoch 4/50
1739/1739 [==============================] - 1s 727us/step - loss: 0.6169 - accuracy: 0.6517 - val_loss: 0.6141 - val_accuracy: 0.6550
Epoch 5/50
1739/1739 [==============================] - 1s 725us/step - loss: 0.6150 - accuracy: 0.6526 - val_loss: 0.6302 - val_accuracy: 0.6317
Epoch 6/50
1739/1739 [==============================] - 1s 726us/step - loss: 0.6133 - accuracy: 0.6540 - val_loss: 0.6103 - val_accuracy: 0.6554
Epoch 7/50
1739/1739 [==============================] - 1s 728us/step - loss: 0.6113 - accuracy: 0.6568 - val_loss: 0.6086 -

In [70]:
model.evaluate(final_pipeline.transform(X_test), y_test)

2028/2028 [==============================] - 1s 332us/step - loss: 0.5956 - accuracy: 0.6735


[0.5956400632858276, 0.6735109090805054]

Get version of the packages used in the pipeline so we could then install all those packages in virtual env

In [ ]:
# print(f'joblib=={joblib.__version__}')
# print(f'scikit-learn=={sklearn.__version__}')
# print(f'category_encoders=={category_encoders.__version__}')